### Lab 2: Hyponyms and Hypernyms

In [2]:
import pandas as pd
import wikipedia
import multiprocessing
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
from ipywidgets import interact, interactive, fixed, interact_manual,widgets
from IPython.display import display
import json
num_cores = multiprocessing.cpu_count()
print(num_cores)
wikipedia.set_lang("ru")
# DATA_PATH_LIST = ['D:','src2','taxonomy-enrichment','data','training_data']
DATA_PATH_LIST = ['.']
EMBEDDING_MODEL_FILENAME = "wiki_node2vec.bin"
DATA_PATH="/".join(DATA_PATH_LIST+["training_nouns.tsv"])
df = pd.read_csv(DATA_PATH,sep='\t')
%matplotlib inline

4


In [ ]:
def prestr(x):
    return str(x).replace('\"','').replace("'",'"')

In [ ]:
class DefDict(defaultdict):
    def __missing__(self, key):
        self[key] = key
        return key
    
idx2syns = DefDict(lambda x:x)
for val in df.values:
    idx2syns[val[0]]=val[1]
    try:
        pidxs = json.loads(prestr(val[2]))
        concp = [el.split(",")[0] for el in json.loads(prestr(val[3]))]
        idx2syns.update(dict(zip(pidxs,concp)))
    except:
        print(prestr(val[2]))
        print(prestr(val[3]))

### Interactive visualization of hyponyms and hypernyms

In [ ]:
button = widgets.Button(description="Draw")
query = widgets.Text(
    value='МАТЬ',
    placeholder='Query',
    description='String:',
    disabled=False
)
display(button,query)




def creategraph(df):
    res = []
    for row in df.values:
        cohyps = row[1].split(",")
        for idx,cohyp in enumerate(cohyps):
            for parent in json.loads(prestr(row[2])):
                res.append((row[0]+'-'+str(idx),parent))
    return res

def graphdraw(b):
    print("graphdraw",query.value)
    subset = df[df['TEXT'].str.contains(query.value.upper())]
    g = nx.DiGraph()
    for el in subset.values:
        cohyps = el[1].split(",")
        print(cohyps)
        syns = idx2syns[el[0]]
        for child in cohyps:
            for parent in json.loads(prestr(el[2])):
                ed = g.add_edge(child,idx2syns[parent],label="is a")
            
    plt.figure(figsize=(15,15))
    pos = nx.nx_agraph.graphviz_layout(g)
    nx.draw(g,with_labels=True,pos=pos)
#     edge_labels=nx.draw_networkx_edge_labels(g,pos=pos)
    plt.show()
button.on_click(graphdraw)

### Pattern extractor

Yargy — библиотека для извлечения структурированной информации из текстов на русском языке. Правила описываются контекстно-свободными грамматиками и словарями ключевых слов. Банк готовых правил для имён, дат, адресов и других сущностей доступен в репозитории Natasha.
* https://yargy.readthedocs.io/ru/latest/
* http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
* https://github.com/natasha/natasha

### Токенизатор

In [3]:
from yargy.tokenizer import MorphTokenizer


tokenizer = MorphTokenizer()
text = '''Ростов-на-Дону
Длительностью 18ч. 10мин.
Яндекс.Такси
π ≈ 3.1415
1 500 000$
http://vk.com
'''
for line in text.splitlines():
    print([_.value for _ in tokenizer(line)])

['Ростов', '-', 'на', '-', 'Дону']
['Длительностью', '18', 'ч', '.', '10', 'мин', '.']
['Яндекс', '.', 'Такси']
['π', '≈', '3', '.', '1415']
['1', '500', '000', '$']
['http', ':', '/', '/', 'vk', '.', 'com']


# Газеттир
Газеттир нужен для удобной работы с последовательностью слов. Например, можно написать:

In [4]:
from yargy import or_, rule
from yargy.predicates import normalized

RULE = or_(
    rule(normalized('dvd'), '-', normalized('диск')),
    rule(normalized('видео'), normalized('файл'))
)

In [5]:
from yargy import Parser
from yargy.pipelines import morph_pipeline


RULE = morph_pipeline([
    'dvd-диск',
    'видео файл',
    'видеофильм',
    'газета',
    'электронный дневник',
    'эссе',
])

parser = Parser(RULE)
text = 'Видео файл на dvd-диске'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Видео', 'файл']
['dvd', '-', 'диске']


In [6]:
from yargy import Parser, rule, and_
from yargy.predicates import gram, is_capitalized, dictionary


GEO = rule(
    and_(
        gram('ADJF'),  # так помечается прилагательное, остальные пометки описаны в
                       # http://pymorphy2.readthedocs.io/en/latest/user/grammemes.html
        is_capitalized()
    ),
    gram('ADJF').optional().repeatable(),
    dictionary({
        'федерация',
        'республика'
    })
)


parser = Parser(GEO)
text = '''
В Чеченской республике на день рождения ...
Донецкая народная республика провозгласила ...
Башня Федерация — одна из самых высоких ...
'''
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

['Чеченской', 'республике']
['Донецкая', 'народная', 'республика']


### Предикаты

Предикат — функция, которая принимает на вход токен и возвращает True или False. В Yargy встроено много готовых предикатов. Полный список есть в справочнике. Предикаты комбинируются с помощью and_, or_ и not_.

In [7]:
from yargy import and_, not_
from yargy.tokenizer import MorphTokenizer
from yargy.predicates import is_capitalized, eq


tokenizer = MorphTokenizer()
token = next(tokenizer('Стали'))

predicate = is_capitalized()
print(predicate(token))

predicate = and_(
    is_capitalized(),
    not_(eq('марки'))
)
print(predicate(token))

True
True


### Грамматики
В Yargy используется специальный DSL для описания грамматик. Любую контекстно-свободную грамматику можно описать с помощью конструкций Питона. Например, есть примитивная грамматика для размеров одежды:

KEY -> р. | размер

VALUE -> S | M | L

SIZE -> KEY VALUE



In [8]:
from yargy import rule, or_


KEY = or_(
    rule('р', '.'),
    rule('размер')
).named('KEY')
VALUE = or_(
    rule('S'),
    rule('M'),
    rule('L'),
    rule('XS'),
).named('VALUE')
SIZE = rule(
    KEY,
    VALUE
).named('SIZE')
SIZE.normalized.as_bnf

SIZE -> KEY VALUE
KEY -> 'р' '.' | 'размер'
VALUE -> 'S' | 'M' | 'L' | 'XS'


In [ ]:
parser = Parser(
    SIZE
)
text = 'размер M; размер A; размер XS;'
for match in parser.findall(text):
    print([_.value for _ in match.tokens])

In [9]:
from yargy import Parser, rule, and_, or_, not_
from yargy.interpretation import fact, attribute
from yargy.predicates import gram, is_capitalized, dictionary, eq
import re
import pandas as pd

from gensim import utils

In [10]:

START = rule(
    or_(
        rule(gram('ADJF')),
        rule(gram('NOUN'))
    ).optional(),
    gram('NOUN')
)

START_S = or_(
    eq('такой'),
    eq('такие'),
)

KAK = eq('как')
INCLUDING = or_(
    or_(
        eq('в'),
        eq('том'),
        eq('числе'),
    ),
    eq('включающий'),
    or_(
        eq('включающий'),
        eq('в'),
        eq('себя'),
    ),
    or_(
        eq('включающие'),
        eq('в'),
        eq('себя'),
    ),
    eq('включающие'),
    eq('особенно'),

)

MID_S = or_(
    rule(
        or_(
            eq('такой'),
            eq('такие'),
        ),
        eq('как')
    )
)
ATAKJE = rule(
    eq(','),
    eq('а'),
    eq('также')
)

MID = or_(
    rule(
        eq('это')
    ),
    rule(
        eq('—')
    ),
    rule(
        eq('—'),
        eq('это')
    ),
    rule(
        eq('—'),
        not_(eq('км'))
    ),
    rule(
        or_(
            eq('и'),
            eq('или'),
        ),
        eq('другие')
    )
)

END = or_(
    rule(
        gram('NOUN'),
        gram('NOUN')
    ),
    rule(
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('ADJF'),
        gram('ADJF').repeatable(),
        gram('NOUN')
    ),
    rule(
        gram('NOUN').repeatable(),
        gram('ADJF'),
        gram('NOUN').repeatable()
    ),
    rule(
        gram('NOUN').repeatable()
    )
)

Item = fact(
    'Item',
    [attribute('titles').repeatable()]
)


IGNORE = rule(
    '(',
    not_(eq(')')).repeatable(),
    ')'
)

ITEM = rule(
    IGNORE.interpretation(
        Item.titles
    ),
    eq(',').optional() 
).repeatable().interpretation(
    Item
)

In [ ]:
def get_hyperonyms(main_word):
    HYPONYM = eq(utils.deaccent(main_word))
    RULE = or_(
        rule(HYPONYM, ATAKJE, START, MID, END),
        rule(HYPONYM, MID, END),
        rule(START_S, END, KAK, HYPONYM),
        rule(END, INCLUDING, HYPONYM)
    )
    parser = Parser(RULE) 
    text = utils.deaccent(wikipedia.summary(main_word))
    print(text)
    text = re.sub(r'\(.+?\)', '', text)
    text = text.lower().replace('* сергии радонежскии* ', '')
    for idx, match in enumerate(parser.findall(text.lower())):
        k = [_.value for _ in match.tokens]
        print(k)

In [ ]:
get_hyperonyms("банан")

#### Task 1 (deadline 19.02.2020 24:00)
* Find your name on the spreadsheet https://docs.google.com/spreadsheets/d/1RR2I6toCkebbGU1UK83HS70Ru_l0_o-nnZIHyiFB0No/edit?usp=sharing. In opposite of your name there are 24 words of hyponyms, you have to insert five corresponding hypernyms next to them. Examples of hyponyms and hyperonyms relationship you can find above in the current Jupiter notebook.
* Find for each pair of hyponyms and hypernyms a corresponding snippet of a text with their mentions. The source of the text can be any free resources, e.g., Wikipedia, Google, Yandex, others. You should save the snippets and their URLs within the lab2 folder in your NLP git-repo with .csv file-extension in a single file.

#### Task 2 (deadline 26.02.2020 24:00)
* It would be best if you created a pandas DataFrame of the texts from the previous task. And apply to the DataFrame the function 'get_hyperonyms,' which must return the list of the corresponding hypernyms from the text automatically. If there are errors or misses, you should fix them in the code for your case of the 24 words. Nevertheless, it is strictly prohibited to use hard coding. Save your notebook with parser code within the lab2 folder in your NLP git-repo.

In [ ]:
# homework_data = pd.read_csv('words_2.csv')

# homework_data


# # for key in keys:
# #     for word in 

# # for row in homework_data.iterrows():
# #     print(row[1].keys())

In [ ]:
# def find_text(word): 
#     return utils.deaccent(wikipedia.summary(word))

In [ ]:
# result = []
# # Search in wiki only HYPERONYMs
# for hypo, hypers in zip(list(homework_data['HYPONYM']), list(homework_data['HYPERONYM'])):
#     hypers = list(map(lambda x: x.strip(), hypers.split(", ")))
#     for hyper in hypers:
#         tmp = {}
#         tmp["hypo"] = hypo
#         if hyper == "":
#             continue
#         tmp["key"] = hyper
#         try:
#             tmp["text"] = find_text(hyper)
#         except:
#             tmp["text"] = ""
#             print("error")
#         result.append(tmp)


In [ ]:
# result[0]

In [ ]:
# import csv
# csv_file = "text_HYPONYM.csv"
# try:
#     with open(csv_file, 'w') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=["hypo", "key", "text"])
#         for data in result:
#             writer.writerow(data)
# except IOError:
#     print("I/O error")

In [ ]:
# def get_hyperonyms2(main_word, text):
#     HYPONYM = eq(utils.deaccent(main_word))
#     RULE = or_(
#         rule(HYPONYM, ATAKJE, START, MID, END),
#         rule(HYPONYM, MID, END),
#         rule(START_S, END, KAK, HYPONYM),
#         rule(END, INCLUDING, HYPONYM)
#     )
#     parser = Parser(RULE) 
# #     text = utils.deaccent(wikipedia.summary(main_word))
# #     print(text)
#     text = re.sub(r'\(.+?\)', '', text)
# #     text = text.lower().replace('* сергии радонежскии* ', '')
#     for idx, match in enumerate(parser.findall(text.lower())):
#         k = [_.value for _ in match.tokens]
#         print(k)

In [ ]:
# for data in result:
#     print('Гиперонимы для слова: ', data["key"])
#     if data["text"] == "":
#         print("Пусто")
#         continue
#     print('\n')
#     get_hyperonyms2(data["key"], data["text"])
#     print('\n\n')

In [21]:
from google import google

class GoogleSearch(object):
    def __init__(self, hypo, hyper, num_page):
        self.hypo = hypo
        self.hyper = hyper
        self.num_page = num_page
        self.res = []
        
    def get_text(self):
        if  len(self.res) > 0:
            return "\n".join(list(map(lambda x: str(x.description), self.res)))
        return None
    
    def get_link(self):
        if  len(self.res) > 0:
            return "\n".join(list(map(lambda x: str(x.link), self.res)))
        return None
    
    def get_google_link(self):
        if  len(self.res) > 0:
            return "\n".join(list(map(lambda x: str(x.google_link), self.res)))
        return None
    
    def find_text(self):
        search_results = google.search(self.hypo + " " + self.hyper, self.num_page)
        print(self.hypo + " " + self.hyper)
        print("Search result count: ", len(search_results))
        for res in search_results:
            parser = HyperParser(self.hypo, self.hyper, res.description)
            parser.get_hyperonyms3()            
            if len(parser.hyper_text) > 0: 
                self.res.append(res)
                

In [12]:
class HyperParser(object):
    def __init__(self, hypo, hyper, text):
        self.hypo = hypo.capitalize()
        self.hyper = hyper
        self.text = text
        self.hyper_text = []
        
    def get_hyperonyms3(self):
        HYPONYM = eq(utils.deaccent(self.hyper))
        RULE = or_(
            rule(HYPONYM, ATAKJE, START, MID, END),
            rule(HYPONYM, MID, END),
            rule(START_S, END, KAK, HYPONYM),
            rule(END, INCLUDING, HYPONYM),
            rule(HYPONYM, MID, START, START),
            rule(HYPONYM, MID, START),
            rule(HYPONYM, END)
        )
        parser = Parser(RULE) 
        _text = re.sub(r'\(.+?\)', '', utils.deaccent(self.text))
        for idx, match in enumerate(parser.findall(_text.lower())):
            k = [_.value for _ in match.tokens]
            if k is not None:
                print(k)
                self.hyper_text.append(k)
                

In [26]:
tmp = GoogleSearch("ГЕККОН", "ящерица", 1)
tmp.find_text()
tmp.get_text()
tmp.get_link()
tmp.get_google_link()

ГЕККОН ящерица
Search result count:  9
['ящерица', 'из', 'семеиства', 'гекконов']
['ящерица', 'рептилия', 'животных', 'зеленыи', 'тварь']
['ящерица', 'геккона']


'None\nNone\nNone'

In [23]:
homework_data = pd.read_csv('words_2.csv')

homework_data

result = []
for hypo, hypers in zip(list(homework_data['HYPONYM']), list(homework_data['HYPERONYM'])):
    hypers = list(map(lambda x: x.strip(), hypers.split(", ")))
    for hyper in hypers:
        tmp = {}
        pair = hypo + " " + hyper
        tmp["hypo"] = hypo
        if hyper == "":
            continue
        tmp["key"] = hyper
        search = GoogleSearch(hypo, hyper, 1)
        try:
            search.find_text()
            if search.get_text() is not None:
                tmp["text"] = search.get_text()
                tmp["link"] = search.get_link()
                tmp["google_link"] = search.get_google_link()                
        except Exception as err:
            tmp["text"] = ""
            tmp["link"] = search.get_link()
            tmp["google_link"] = search.get_google_link()                
            print(pair)
            print(err)
        result.append(tmp)

ГЕККОН ящерица
Search result count:  9
['ящерица', 'из', 'семеиства', 'гекконов']
['ящерица', 'рептилия', 'животных', 'зеленыи', 'тварь']
['ящерица', 'геккона']
ГЕККОН чешуйчатый
Search result count:  9
['чешуичатыи', 'покров', 'головы', 'средиземноморского', 'геккона']
['чешуичатыи', 'покров', 'головы', 'средиземноморского', 'геккона']
ГЕККОН пресмыкающиеся
Search result count:  7
ГЕККОН животное
Search result count:  8
['животное', 'под']
ГЕККОН рептилия
Search result count:  0
ГЕЛЬ желеобразное вещество
Search result count:  9
ГЕЛЬ косметика
Search result count:  9
['косметика', 'алоэ']
['косметика', 'гель', 'алоэ']
['косметика', 'в', 'украине']
ГЕОРГИН растение
Search result count:  10
['растение', 'георгина', 'в', 'честь', 'известного', 'ботаника']
['растение', 'с']
['растение', 'с']
ГЕОРГИН цветы
Search result count:  8
['цветы', 'георгины', 'семеиства']
['цветы', 'георгины']
['цветы', 'георгины']
ГЕОРГИН сложноцветный цветок
Search result count:  9
ГЕОРГИН трава
Search result co

In [24]:
result[0]

{'hypo': 'ГЕККОН',
 'key': 'ящерица',
 'text': 'Геккон токи (лат. Gekko gecko) — ящерица из семейства гекконов. Содержание. 1 Описание; 2 Ареал и места обитания; 3 Питание; 4 Размножение\xa0...\nОчистить фильтры. Похожие изображения: ящерица рептилия животных зеленый тварь. 376 Бесплатные фото Геккон. Геккон, Рептилия, Террариум\xa0...\nСкачать стоковое фото ящерица геккона ✓ популярный фотобанк ✓ доступные цены ✓ миллионы роялти-фри фотографий, изображений и картинок в\xa0...',
 'link': 'https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D0%BA%D0%BA%D0%BE%D0%BD_%D1%82%D0%BE%D0%BA%D0%B8\nhttps://pixabay.com/ru/photos/search/%D0%B3%D0%B5%D0%BA%D0%BA%D0%BE%D0%BD/\nhttps://ru.depositphotos.com/stock-photos/%D1%8F%D1%89%D0%B5%D1%80%D0%B8%D1%86%D0%B0-%D0%B3%D0%B5%D0%BA%D0%BA%D0%BE%D0%BD%D0%B0.html',
 'google_link': 'None\nNone\nNone'}

In [25]:
import csv
csv_file = "text_HYPONYM.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["hypo", "key", "text", "link", "google_link"])
        for data in result:
            writer.writerow(data)
except IOError:
    print("I/O error")